In [1]:
import httpx
import polars as pl

In [2]:
INE_BASE_URL = "https://servicios.ine.es/wstempus/js/ES"

client = httpx.Client(
    base_url=INE_BASE_URL,
    limits=httpx.Limits(max_keepalive_connections=20),
    transport=httpx.HTTPTransport(retries=3),
)

In [3]:
def ine_request(client: httpx.Client, endpoint, paginate=True):
    page = 1
    data = []

    while True:
        params = {"det": 10}

        if paginate:
            params["page"] = page

        response = client.get(
            f"/{endpoint}", params=params, follow_redirects=True
        ).json()

        if not response:
            break

        data.extend(response)

        if len(response) < 500 or not paginate:
            break

        page += 1

    return data

In [4]:
# https://servicios.ine.es/wstempus/js/ES/OPERACIONES_DISPONIBLES

operaciones_disponibles = pl.DataFrame(ine_request(client, "OPERACIONES_DISPONIBLES"))
print(operaciones_disponibles.shape)
operaciones_disponibles.sample(5)

(110, 6)


Id,Cod_IOE,Nombre,Codigo,Referencia,Url
i64,str,str,str,list[struct[3]],str
353,"""30325""","""Atlas de distribución de renta…","""ADRH""","[{197,""Atlas de distribución de renta de los hogares"",""/dyngs/INEbase/es/operacion.htm?c=Estadistica_C&cid=1254736177088&idp=1254735976608""}]","""https://www.ine.es/dyngs/INEba…"
254,"""30220""","""Índice de ingresos hoteleros""","""IIH""",null,null
25,"""30138""","""Índice de Precios de Consumo (…","""IPC""","[{65,""Índice de precios de consumo"",""/dyngs/INEbase/es/operacion.htm?c=Estadistica_C&cid=1254736176802&idp=1254735976607""}]",null
437,"""""","""Medición del turismo emisor a …","""TMOV""",null,"""https://www.ine.es/experimenta…"
240,"""30237""","""Encuesta de Ocupación en Campi…","""EOAC""","[{114,""Campings: encuesta de ocupación e índice de precios"",""/dyngs/INEbase/es/operacion.htm?c=Estadistica_C&cid=1254736176961&idp=1254735576863""}]",null


In [5]:
# https://servicios.ine.es/wstempus/js/ES/VARIABLES?det=10

variables = pl.DataFrame(ine_request(client, "VARIABLES"))
variables.sample(5)

Id,Nombre,Codigo
i64,str,str
304,"""Destino económico de los biene…",""""""
918,"""% VN que representan las OAL s…",""""""
666,"""Tiempo transcurrido""",""""""
249,"""Cifra de Negocio/Personal Ocup…",""""""
142,"""Zonas de nacionalidad""",""""""


In [6]:
t = []

for row in operaciones_disponibles.rows(named=True):
    tablas_operacion_url = f"TABLAS_OPERACION/{row['Id']}"
    tablas_operacion = ine_request(client, tablas_operacion_url)

    t.extend(tablas_operacion)

In [7]:
import os

os.makedirs("dataset", exist_ok=True)

tablas = pl.json_normalize(t)
print(tablas.shape)
tablas.write_ndjson("dataset/tablas.jsonl")
tablas.sample(5)

(5030, 51)


Id,Nombre,Codigo,Anyo_Periodo_ini,FechaRef_fin,Ultima_Modificacion,Periodicidad.Id,Periodicidad.Nombre,Periodicidad.Codigo,Publicacion.Id,Publicacion.Nombre,Publicacion.Periodicidad.Id,Publicacion.Periodicidad.Nombre,Publicacion.Periodicidad.Codigo,Publicacion.Operacion,Publicacion.PubFechaAct.Id,Publicacion.PubFechaAct.Nombre,Publicacion.PubFechaAct.Fecha,Publicacion.PubFechaAct.Periodo.Id,Publicacion.PubFechaAct.Periodo.Valor,Publicacion.PubFechaAct.Periodo.Periodicidad.Id,Publicacion.PubFechaAct.Periodo.Periodicidad.Nombre,Publicacion.PubFechaAct.Periodo.Periodicidad.Codigo,Publicacion.PubFechaAct.Periodo.Dia_inicio,Publicacion.PubFechaAct.Periodo.Mes_inicio,Publicacion.PubFechaAct.Periodo.Codigo,Publicacion.PubFechaAct.Periodo.Nombre,Publicacion.PubFechaAct.Periodo.Nombre_largo,Publicacion.PubFechaAct.Anyo,Periodo_ini.Id,Periodo_ini.Valor,Periodo_ini.Periodicidad.Id,Periodo_ini.Periodicidad.Nombre,Periodo_ini.Periodicidad.Codigo,Periodo_ini.Dia_inicio,Periodo_ini.Mes_inicio,Periodo_ini.Codigo,Periodo_ini.Nombre,Periodo_ini.Nombre_largo,Publicacion.Url,Anyo_Periodo_fin,Periodo_fin.Id,Periodo_fin.Valor,Periodo_fin.Periodicidad.Id,Periodo_fin.Periodicidad.Nombre,Periodo_fin.Periodicidad.Codigo,Periodo_fin.Dia_inicio,Periodo_fin.Mes_inicio,Periodo_fin.Codigo,Periodo_fin.Nombre,Periodo_fin.Nombre_largo
i64,str,str,str,str,i64,i64,str,str,i64,str,i64,str,str,list[struct[6]],i64,str,i64,i64,i64,i64,str,str,str,str,str,str,str,i64,i64,i64,i64,str,str,str,str,str,str,str,str,str,i64,i64,i64,str,str,str,str,str,str,str
66236,"""Asalariados por tipo de jornad…","""NAC""","""2006""",null,1478858400000,12,"""Anual""","""A""",348,"""Encuesta de población activa. …",12,"""Anual""","""A""","[{293,""30308"",""Encuesta de Población Activa (EPA)"",""EPA"",null,[{138,""Encuesta de población activa"",""/dyngs/INEbase/es/operacion.htm?c=Estadistica_C&cid=1254736176918&idp=1254735976595""}]}]",10640,"""Encuesta de población activa. …",1732269600000,28,1,12,"""Anual""","""A""","""1""","""1""","""01""","""A""","""Año""",2023,28,1,12,"""Anual""","""A""","""1""","""1""","""01""","""A""","""Año""","""https://ine.es/dyngs/INEbase/e…","""2015""",28,1,12,"""Anual""","""A""","""1""","""1""","""01""","""A""","""Año"""
30904,"""Indicadores demográficos""","""MUN-DIST-SECC""","""2015""","""null""",1730196000000,12,"""Anual""","""A""",507,"""Atlas de distribución de renta…",12,"""Anual""","""A""","[{353,""30325"",""Atlas de distribución de renta de los hogares"",""ADRH"",""https://www.ine.es/dyngs/INEbase/es/operacion.htm?c=Estadistica_C&cid=1254736177088&idp=1254735976608"",[{197,""Atlas de distribución de renta de los hogares"",""/dyngs/INEbase/es/operacion.htm?c=Estadistica_C&cid=1254736177088&idp=1254735976608""}]}]",10582,"""Atlas de distribución de renta…",1730196000000,28,1,12,"""Anual""","""A""","""1""","""1""","""01""","""A""","""Año""",2022,28,1,12,"""Anual""","""A""","""1""","""1""","""01""","""A""","""Año""","""https://www.ine.es/dyngs/INEba…",null,null,null,null,null,null,null,null,null,null,null
66175,"""Asalariados por frecuencia con…","""NAC-CCAA""","""2006""","""null""",1679652000000,12,"""Anual""","""A""",347,"""Encuesta de Población Activa. …",12,"""Anual""","""A""","[{293,""30308"",""Encuesta de Población Activa (EPA)"",""EPA"",null,[{138,""Encuesta de población activa"",""/dyngs/INEbase/es/operacion.htm?c=Estadistica_C&cid=1254736176918&idp=1254735976595""}]}]",10632,"""Encuesta de Población Activa. …",1713517200000,28,1,12,"""Anual""","""A""","""1""","""1""","""01""","""A""","""Año""",2023,28,1,12,"""Anual""","""A""","""1""","""1""","""01""","""A""","""Año""",null,null,null,null,null,null,null,null,null,null,null,null
20517,"""Separaciones entre cónyuges de…","""CCAA""","""2013""","""null""",1720688400000,12,"""Anual""","""A""",106,"""Estadística de nulidades, sepa…",12,"""Anual""","""A""","[{66,""30463"",""Estadística de Nulidades, Separaciones y Divorcios"",""ENSD"",null,[{166,""Estadística de nulidades, separaciones y divorcios"",""/

In [8]:
def generate_readme_content(data):
    return f"""# {data["Nombre"]}

- **ID:** [{data["Id"]}](https://www.ine.es/jaxiT3/Tabla.htm?t={data["Id"]})
- **Código:** {data["Codigo"]}
- **Año de Inicio del Periodo:** {data["Anyo_Periodo_ini"]}
- **Fecha de Referencia Final:** {data["FechaRef_fin"]}
- **Última Modificación:** {data["Ultima_Modificacion"]}
- **Periodicidad:** {data["Periodicidad.Nombre"]} ({data["Periodicidad.Codigo"]})

## Publicación

- **ID:** {data["Publicacion.Id"]}
- **Nombre:** {data["Publicacion.Nombre"]}
- **Periodicidad:** {data["Publicacion.Periodicidad.Nombre"]} ({data["Publicacion.Periodicidad.Codigo"]})

### Operación

- **ID:** {data["Publicacion.Operacion"][0]["Id"]}
- **Código IOE:** {data["Publicacion.Operacion"][0]["Cod_IOE"]}
- **Nombre:** {data["Publicacion.Operacion"][0]["Nombre"]}
- **Referencia:** {data["Publicacion.Operacion"][0]["Referencia"]}

### Última Publicación

- **ID:** {data["Publicacion.PubFechaAct.Id"]}
- **Nombre:** {data["Publicacion.PubFechaAct.Nombre"]}
- **Fecha:** {data["Publicacion.PubFechaAct.Fecha"]}
- **Periodo ID:** {data["Publicacion.PubFechaAct.Periodo.Id"]}
- **Periodo Valor:** {data["Publicacion.PubFechaAct.Periodo.Valor"]}
- **Periodo Periodicidad:** {data["Publicacion.PubFechaAct.Periodo.Periodicidad.Nombre"]} ({data["Publicacion.PubFechaAct.Periodo.Periodicidad.Codigo"]})
- **Periodo Día de Inicio:** {data["Publicacion.PubFechaAct.Periodo.Dia_inicio"]}
- **Periodo Mes de Inicio:** {data["Publicacion.PubFechaAct.Periodo.Mes_inicio"]}
- **Periodo Código:** {data["Publicacion.PubFechaAct.Periodo.Codigo"]}
- **Periodo Nombre:** {data["Publicacion.PubFechaAct.Periodo.Nombre"]}
- **Periodo Nombre Largo:** {data["Publicacion.PubFechaAct.Periodo.Nombre_largo"]}
- **Año:** {data["Publicacion.PubFechaAct.Anyo"]}

## Periodo Inicial

- **ID:** {data["Periodo_ini.Id"]}
- **Valor:** {data["Periodo_ini.Valor"]}
- **Periodicidad:** {data["Periodo_ini.Periodicidad.Nombre"]} ({data["Periodo_ini.Periodicidad.Codigo"]})
- **Día de Inicio:** {data["Periodo_ini.Dia_inicio"]}
- **Mes de Inicio:** {data["Periodo_ini.Mes_inicio"]}
- **Código:** {data["Periodo_ini.Codigo"]}
- **Nombre:** {data["Periodo_ini.Nombre"]}
- **Nombre Largo:** {data["Periodo_ini.Nombre_largo"]}

## Periodo Final

- **Año de Fin del Periodo:** {data["Anyo_Periodo_fin"]}
- **ID:** {data["Periodo_fin.Id"]}
- **Valor:** {data["Periodo_fin.Valor"]}
- **Periodicidad:** {data["Periodo_fin.Periodicidad.Nombre"]} ({data["Periodo_fin.Periodicidad.Codigo"]})
- **Día de Inicio:** {data["Periodo_fin.Dia_inicio"]}
- **Mes de Inicio:** {data["Periodo_fin.Mes_inicio"]}
- **Código:** {data["Periodo_fin.Codigo"]}
- **Nombre:** {data["Periodo_fin.Nombre"]}
- **Nombre Largo:** {data["Periodo_fin.Nombre_largo"]}

## JSON

```json
{data}
```
"""

In [9]:
def get_series_tabla_url(tabla_id):
    return (
        f"https://servicios.ine.es/wstempus/jsCache/ES/SERIES_TABLA/{tabla_id}?det=10"
    )


def get_tablas_download_url(tabla_id):
    return f"https://www.ine.es/jaxiT3/files/t/es/csv_bdsc/{tabla_id}.csv"


tablas = tablas.with_columns(
    pl.col("Id")
    .map_elements(get_series_tabla_url, return_dtype=pl.String)
    .alias("series_tabla_url"),
    pl.col("Id")
    .map_elements(get_tablas_download_url, return_dtype=pl.String)
    .alias("tablas_download_url"),
)

In [10]:
# tablas = tablas.sample(10)

In [11]:
with open("series.input.spec", "w") as f:
    for t in tablas.rows(named=True):
        f.write(f"{t['series_tabla_url']}\n")
        f.write("\tout=series_metadata.json\n")
        f.write(f"\tdir=dataset/tablas/{t['Id']}\n\n")

with open("tablas.input.spec", "w") as f:
    for t in tablas.rows(named=True):
        f.write(f"{t['tablas_download_url']}\n")
        f.write(f"\tout={t['Id']}.csv\n")
        f.write(f"\tdir=dataset/tablas/{t['Id']}\n\n")

In [12]:
import subprocess

series_result = subprocess.run(
    [
        "aria2c",
        "-i",
        "series.input.spec",
        "-j",
        "50",
        "-x",
        "16",
        "-s",
        "8",
        "-c",
        "--file-allocation=none",
        "--console-log-level=warn",
    ],
    capture_output=True,
    text=True,
)

KeyboardInterrupt: 

In [ ]:
print(series_result.stdout)

In [13]:
result = subprocess.run(
    ["grep", "-rl", "unos minutos", "dataset/tablas"],
    capture_output=True,
    text=True,
)

missing_metadata_files = result.stdout.splitlines()

print(f"Missing metadata files: {len(missing_metadata_files)}")

Missing metadata files: 3


In [14]:
with open("missing_series.input.spec", "w") as f:
    for file in missing_metadata_files:
        tabla_id = file.split("/")[-2]

        f.write(
            f"https://servicios.ine.es/wstempus/jsCache/ES/SERIES_TABLA/{tabla_id}?det=10\n"
        )
        f.write("\tout=metadata.json\n")
        f.write(f"\tdir=dataset/tablas/{tabla_id}\n\n")


In [15]:
missing_series_result = subprocess.run(
    [
        "aria2c",
        "-i",
        "missing_series.input.spec",
        "-j",
        "50",
        "-x",
        "16",
        "-s",
        "8",
        "-c",
        "--file-allocation=none",
        "--console-log-level=warn",
        "--allow-overwrite=true",
    ],
    capture_output=True,
    text=True,
)

In [ ]:
print(missing_series_result.stdout)

In [17]:
tablas_result = subprocess.run(
    [
        "aria2c",
        "-i",
        "tablas.input.spec",
        "-j",
        "50",
        "-x",
        "16",
        "-s",
        "8",
        "-c",
        "--file-allocation=none",
        "--console-log-level=warn",
        "--allow-overwrite=true",
    ],
    capture_output=True,
    text=True,
)

In [ ]:
print(tablas_result.stdout)

In [31]:
import glob

from tqdm import tqdm

csv_files = glob.glob("dataset/tablas/*/*.csv")

for file in tqdm(csv_files):
    filename = file.split(".")[-2].split("/")[-1]

    table_metadata = tablas.filter(pl.col("Id") == int(filename)).to_struct()

    if table_metadata.shape[0] > 0:
        with open(f"dataset/tablas/{filename}/README.md", "w") as f:
            table_metadata = table_metadata[0]
            f.write(generate_readme_content(table_metadata))

    (
        pl.scan_csv(
            file,
            separator=";",
            ignore_errors=True,
            truncate_ragged_lines=True,
        ).sink_parquet(
            f"dataset/tablas/{filename}/datos.parquet",
            compression="zstd",
            row_group_size=1024**2,
            type_coercion=True,
        )
    )


100%|██████████| 4843/4843 [03:41<00:00, 21.89it/s] 


In [32]:
import os

for file in csv_files:
    os.remove(file)

In [ ]:
import os

from huggingface_hub import HfApi

api = HfApi(token=os.getenv("HUGGINGFACE_TOKEN"))

In [ ]:
import shutil

shutil.copy("DATASET_README.md", "dataset/README.md")

api.upload_large_folder(
    folder_path="dataset", repo_id="davidgasquez/ine", repo_type="dataset"
)